# [06] COVID Activation Map의 시각화

본 실습에서는 학습한 이미지 분류 신경망의 응용 중 하나인 `Activation Map의 시각화`에 대해서 다뤄볼려고 합니다.

아래의 자료는 하기에 표기된 저장소의 자료를 기반으로 만들었습니다.

- Author:   Kazuto Nakashima
- URL:      http://kazuto1011.github.io
- Created:  2017-05-26

## Activation Map

`Activation Map`은 Interpretable Machine Learning에서 많이 사용/언급 되는 기법입니다.

![image-features-prediction-diagram](./imgs/image-features-prediction-diagram.png)

심층 신경망은 주어진 입력 이미지에 대하여, 사전에 학습된 여러 개의 층들을 통과해 feature extraction을 수행하고 해당되는 feature map들을 뽑아냅니다.

그리고 신경망은 그것을 기반으로 하여 예측(prediction)을 합니다.

여기서 Activation map을 본다함은, feature map의 연산에 중요하게 관여하는 부분이 원래 이미지의 어디에서 오는지를 본다~ 입니다.

즉 feature map의 활성화(activation) 부분을 본다입니다. (feature map이라는 용어는 쉽게 신경망을 통과하면서 중간에 나오는 출력값들이라 생각하시면 됩니다.)

하기의 코드들은 Activation Map을 보기위한 방법 중 `Class Activation Map`에 해당됩니다.

### Class Activation Map

![class-activation-mapping](./imgs/class-activation-mapping.png)

Class Activation Map은 컨볼루션 layer 상에서 Attribution을 수행하기 때문에 상대적으로 부드러운 Attribution 결과를 보여준다는 특징이 있습니다. (위 그림)

In [1]:
from collections import Sequence

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm

/home/taehyeon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


## Visualization을 위한 BaseWrapper

In [2]:
class _BaseWrapper(object):
    def __init__(self, model):
        super(_BaseWrapper, self).__init__()
        self.device = next(model.parameters()).device
        self.model = model
        self.handlers = []  # a set of hook function handlers

    def _encode_one_hot(self, ids):
        one_hot = torch.zeros_like(???????).to(self.device)
        one_hot.scatter_(1, ids, 1.0)
        return one_hot

    def forward(self, image):
        self.image_shape = image.shape[2:]
        self.logits = ???????
        self.probs = ???????
        return self.probs.sort(dim=1, descending=True)  # ordered results

    def backward(self, ids):
        """
        Class-specific backpropagation
        """
        one_hot = self._encode_one_hot(ids)
        self.model.zero_grad()
        self.logits.backward(gradient=one_hot, retain_graph=True)

    def generate(self):
        raise NotImplementedError

    def remove_hook(self):
        """
        Remove all the forward/backward hook functions
        """
        for handle in self.handlers:
            handle.remove()

## BackPropagation

이미지에 걸리는 Gradient를 보고 다른 색이 칠해져 있는 부분이 activation이 강하다고 봅니다.

In [3]:
class BackPropagation(_BaseWrapper):
    def forward(self, image):
        self.image = ??????????
        return super(BackPropagation, self).forward(self.image)

    def generate(self):
        gradient = self.image.grad.clone()
        
        ### make zero_grad
        ????????????
        return gradient

## GuidedBackPropagation

위와 같지만 다른 점이 있다면, relu outputs이 양수인 부분에만 gradients를 구하는 방식입니다.

$$ gradients = \frac{\partial y_{label}}{\partial\, last\, conv\, layer} (gradients>0) \,\& \,(relu\,output>0)$$

In [4]:
class GuidedBackPropagation(BackPropagation):
    """
    "Striving for Simplicity: the All Convolutional Net"
    https://arxiv.org/pdf/1412.6806.pdf
    Look at Figure 1 on page 8.
    """

    def __init__(self, model):
        super(GuidedBackPropagation, self).__init__(model)

        def backward_hook(module, grad_in, grad_out):
            # Cut off negative gradients
            if isinstance(module, nn.ReLU):
                return (????????????)

        for module in self.model.named_modules():
            self.handlers.append(module[1].register_backward_hook(backward_hook))

## GradCAM

Grad-CAM(Gradient-weighted CAM)은 CAM을 구할 때,`예측 이미지안의 중요한 부분을 강조하는 대략적인 지역 맵을 생산하기위한 마지막 컨볼루션 층으로 흘러가는`,
`타겟 클래스(캡션, 마스크도 가능)에 대한` gradient를 이용합니다.

![gradcam_overview](./imgs/gradcam_overview.png)

https://wordbe.tistory.com/entry/Grad-CAMGradient-weighted-Class-Activation-Mapping 의 자료를 참고했습니다.

![grad_cam_1](./imgs/grad_cam_1.png)

Classification 문제에서 예를 들어보면, Grad-CAM( width=u, height=v 인 특정 클래스 c에 대한 이미지 )을 얻기위해

backprop을 통한 gradient 값들을 얻습니다.

이를 위해 softmax 전 단계의 각 클래스에 대한 y score를, k번째 특징 맵 A에 대한 gradient를 얻습니다.

여기에 GAP(Global Average Pooling) 값과 곱하여 뉴런 중요도 가중치(neuron importance weight)인 $$\alpha_k^c$$

를 얻습니다.

![grad_cam_2](./imgs/grad_cam_2.png)

이렇게 얻은 가중치는 타켓 클래스 c에 대한 특징 맵 k의 중요도를 잡을 수 있는데요,

k개의 각 뉴런 중요도 가중치와, 각 특징 맵을 곱하고 더하여 (linear combination) ReLU를 덮어씌웁니다.

 

클래스의 interest에서 양의 값의 영향에 관심이 있기 때문에 렐루를 붙였습니다.

그리고 결과적으로 더 좋은 CAM을 만들었음을 실험으로 밝혔습니다.

### Grad-CAM as a generalization to CAM

![grad_cam_3](./imgs/grad_cam_3.png)

각 클래스에 대한 스코어 S를 얻기 위해 다음과 같은 식을 이용할 수 있습니다.

![grad_cam_4](./imgs/grad_cam_4.png)



In [6]:
class GradCAM(_BaseWrapper):
    """
    "Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization"
    https://arxiv.org/pdf/1610.02391.pdf
    Look at Figure 2 on page 4
    """

    def __init__(self, model, candidate_layers=None):
        super(GradCAM, self).__init__(model)
        self.fmap_pool = {}
        self.grad_pool = {}
        self.candidate_layers = candidate_layers  # list

        def save_fmaps(key):
            def forward_hook(module, input, output):
                self.fmap_pool[key] = output.detach()

            return forward_hook

        def save_grads(key):
            def backward_hook(module, grad_in, grad_out):
                self.grad_pool[key] = grad_out[0].detach()

            return backward_hook

        # If any candidates are not specified, the hook is registered to all the layers.
        for name, module in self.model.named_modules():
            if self.candidate_layers is None or name in self.candidate_layers:
                self.handlers.append(module.register_forward_hook(save_fmaps(name)))
                self.handlers.append(module.register_backward_hook(save_grads(name)))

    def _find(self, pool, target_layer):
        if target_layer in pool.keys():
            return pool[target_layer]
        else:
            raise ValueError("Invalid layer name: {}".format(target_layer))

    def generate(self, target_layer):
        fmaps = self._find(self.fmap_pool, target_layer)
        grads = self._find(self.grad_pool, target_layer)
        weights = F.adaptive_avg_pool2d(grads, 1)

        gcam = torch.mul(fmaps, weights).sum(dim=1, keepdim=True)
        gcam = F.relu(gcam)
        gcam = F.interpolate(
            gcam, self.image_shape, mode="bilinear", align_corners=False
        )

        B, C, H, W = gcam.shape
        gcam = gcam.view(B, -1)
        gcam -= gcam.min(dim=1, keepdim=True)[0]
        gcam /= gcam.max(dim=1, keepdim=True)[0]
        gcam = gcam.view(B, C, H, W)

        return gcam

---------
### <생각해 봅시다>

- Grad-CAM의 gradient는 어디서 얻어 오는 것이 적절할까요?

------------